<a href="https://colab.research.google.com/github/nilesh0109/CV2_SoSe_19/blob/master/ex_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import tensorflow as tf
"""
unpickle function from CIFAR-10 website
"""
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='latin')
    return d

def create_dataset_from_files(files):
	rawdata = []
	labels = []
	for f in files:
		d = unpickle(f)
		rawdata.extend(d["data"])
		labels.extend(d["labels"])

	rawdata = np.array(rawdata)

	red = rawdata[:,:1024].reshape((-1,32,32))
	green = rawdata[:,1024:2048].reshape((-1,32,32))
	blue = rawdata[:,2048:].reshape((-1,32,32))

	data = np.stack((red,green,blue), axis=3)
	labels = np.array(labels)

	return data, labels

cifar_training_files = ['data_batch_{:d}'.format(i) for i in range(1,6)]
cifar_testing_files = ['test_batch']

train_data, train_labels = create_dataset_from_files(cifar_training_files)
test_data, test_labels = create_dataset_from_files(cifar_testing_files)

print(train_data.shape, train_data.dtype) # (50000, 32, 32, 3) -- 50000  images, each 32 x 32 with 3 color channels (UINT8)
print(train_labels.shape, train_labels.dtype) # 50000 labels, one for each image (INT64)

print(test_data.shape, test_data.dtype) # 10000 images, each 32 x 32 with 3 color channels (UINT8)
print(train_labels.shape, train_labels.dtype) # 10000 labels, one for each image (INT64)


(50000, 32, 32, 3) uint8
(50000,) int64
(10000, 32, 32, 3) uint8
(50000,) int64


In [0]:
import tensorflow as tf
import numpy as np

#Training the Neural Network
def train_network(train_data, train_label, batch_size, learning_rate):
    imgs_placeholder = tf.placeholder(tf.uint8, shape=(None, 32, 32, 3))
    imgs_label_placeholder = tf.placeholder(tf.int64, shape=(None,))
    preprocessed_imgs = tf.image.convert_image_dtype(imgs_placeholder, tf.float32)
    
    ######################### first convolutional layer ################################
    conv1_1out = tf.layers.conv2d(preprocessed_imgs, filters=32, kernel_size=(3,3), activation=tf.nn.relu, name='conv1_1')
    conv1_2out = tf.layers.conv2d(conv1_1out, filters=32, kernel_size=(3,3), activation=tf.nn.relu, name='conv1_2')
    pool1_out = tf.layers.max_pooling2d(conv1_2out, pool_size=(2,2), strides=2, name='pool1')
    
    ######################### Second convolutional layer ################################
    conv2_1out = tf.layers.conv2d(pool1_out, filters=64, kernel_size=(3,3), activation=tf.nn.relu, name='conv2_1')
    conv2_2out = tf.layers.conv2d(conv2_1out, filters=64, kernel_size=(3,3), activation=tf.nn.relu, name='conv2_2')
    pool2_out = tf.layers.max_pooling2d(conv2_2out, pool_size=(2,2), strides=2, name='pool2')
    
    ######################### Fully connected layer ################################
    conv2_1out = tf.layers.flatten(pool2_out, name='flatten')
    logits = tf.layers.dense(conv2_1out, units=10, name='fc1')
    
    ######################### Define Loss function ################################
    loss =  tf.losses.sparse_softmax_cross_entropy(imgs_label_placeholder, logits)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
    minimizer_op = optimizer.minimize(loss)
    
    ################## Add ops to save and restore all the variables. ###############
    saver = tf.train.Saver()
    
    ######################### Accuracy check ################################
    predictions = tf.argmax(logits, axis=1)
    correct_pred = tf.equal(imgs_label_placeholder, predictions)
    accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))
    
    ################## Adding summary for consoling through tensorboard ###############
    loss_summary = tf.summary.scalar(name="loss", tensor=loss) #writes loss summary
    acc_summary = tf.summary.scalar(name="accuracy", tensor=accuracy) #writes accuracy summary
    
    ######################### Start Training ################################
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, "model.ckpt")
        print('model restored')
        for k in range(50):
            j = 0
            unseen_data = train_data[:]
            while(len(unseen_data)):
                b_size = batch_size if batch_size < len(unseen_data) else len(unseen_data)
                idx = np.random.choice( unseen_data.shape[0], b_size, replace=False )
                indices_to_keep = [i for i in range(len(unseen_data)) if i not in idx]
                unseen_data = unseen_data[indices_to_keep]
                j = j + 1
                training_data_batch = train_data[idx]
                train_label_batch = train_label[idx]
                _, l, acc, l_summary, a_summary =sess.run([minimizer_op, loss, accuracy, loss_summary, acc_summary], {imgs_placeholder: training_data_batch, imgs_label_placeholder: train_label_batch})
                writer = tf.summary.FileWriter('./graphs', sess.graph)
                writer.add_summary(l_summary, global_step= j) # writes loss summary
                writer.add_summary(a_summary, global_step=j) # writes loss summary
                #print('After iteration {} batch {} loss is {} and accuracy is {}%'.format(k, j, l, acc))
            
            print('*********************************************************************')
            print('After iteration {} loss is {} and accuracy is {}%'.format(k, l, acc*100))
            save_path = saver.save(sess, "model.ckpt".format(k))   
        

    

In [0]:
train_network(train_data, train_labels, 32, 0.001)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model.ckpt
model restored
*********************************************************************
After iteration 0 loss is 0.24775226414203644 and accuracy is 100.0%
*********************************************************************
After iteration 1 loss is 0.196519136428833 and accuracy is 100.0%
*********************************************************************
After iteration 2 loss is 0.22173868119716644 and accuracy is 100.0%
***************************************************************